In [2]:
import numpy as np
import pandas as pd

## Choose your fighter!

In [49]:
df = pd.read_csv("./timings_x86.tsv", sep="\t")

In [55]:
df = pd.read_csv("./timings_avr.tsv", sep="\t")

---

In [57]:
N = 100
kinds = [
    ("compilation_time", "Компиляция программы"),
    ("gdb_start_time", "Запуск процесса GDB"),
    ("container_create_time", "Создание контейнера"),
    ("container_start_time", "Запуск контейнера"),
    ("connect_time", "Подключение к GDB серверу"),
    ("init_breakpoints_time", "Настройка точек останова"),
    ("start_program_time", "Старт программы"),
    ("other", "Прочее"),
    ("total_time", "\\textbf{Итого}"),
]

errs = []

for i in range(N):
    vals = df[df["idx"] == i]["value"].to_numpy()
    accounted_for = sum(vals[:-1])
    total = vals[-1]
    errs.append(total - accounted_for)

errs = np.array(errs)

for kind, name in kinds:
    if kind == "other":
        vals = errs
    else:
        vals = df[df["measurement"] == kind]["value"].to_numpy()
    print(f"{name} & ${np.mean(vals):.3f}$ & ${np.std(vals):.3f}$ \\\\ \\hline".replace(".", "{,}"))

Компиляция программы & $0{,}010$ & $0{,}002$ \\ \hline
Запуск процесса GDB & $0{,}008$ & $0{,}001$ \\ \hline
Создание контейнера & $0{,}027$ & $0{,}002$ \\ \hline
Запуск контейнера & $0{,}946$ & $0{,}069$ \\ \hline
Подключение к GDB серверу & $0{,}203$ & $0{,}000$ \\ \hline
Настройка точек останова & $0{,}001$ & $0{,}000$ \\ \hline
Старт программы & $0{,}001$ & $0{,}000$ \\ \hline
Прочее & $0{,}004$ & $0{,}000$ \\ \hline
\textbf{Итого} & $1{,}200$ & $0{,}069$ \\ \hline


In [133]:
df = pd.read_csv("./timings_single_step_x86.tsv", sep="\t")

In [135]:
df = pd.read_csv("./timings_single_step_avr.tsv", sep="\t")

In [136]:
kinds = [
    ("next_step", "Сделать один шаг"),
    ("set_register_value", "Изменить регистр"),
    ("continue", "Продолжить исполнение"),
]

for kind, name in kinds:
    vals = df[df["measurement"] == kind]["value"].to_numpy()
    print(f"{name} & ${np.mean(vals):.4f}$ & ${np.std(vals):.4f}$ \\\\ \\hline".replace(".", "{,}"))

Сделать один шаг & $0{,}0026$ & $0{,}0003$ \\ \hline
Изменить регистр & $0{,}0034$ & $0{,}0003$ \\ \hline
Продолжить исполнение & $0{,}0036$ & $0{,}0003$ \\ \hline


In [82]:
df = pd.read_csv("./mem_cpu_data.csv")
df["cpu_mean"] /= 100.0
df["cpu_std"] /= 100.0
df["exec_cpu_mean"] /= 100.0
df["exec_cpu_std"] /= 100.0
df

,profile,n,mem_mean,mem_std,cpu_mean,cpu_std,exec_mem_mean,exec_mem_std,exec_cpu_mean,exec_cpu_std
0,Без нагрузки,0,132.2200,0.074922,0.00139,0.000144,0.00,0.000000,0.000000,0.000000
1,SingleStep (x86),10,277.7400,0.444004,0.05740,0.003750,10.45,0.163086,0.009580,0.000476
2,SingleStep (x86),50,854.4000,0.343398,0.26900,0.009210,50.74,0.295674,0.046900,0.001270
3,SingleStep (x86),100,1581.1584,0.301758,0.50600,0.018000,100.59,0.275391,0.089800,0.002470
4,BusyLoop (x86),10,275.3300,0.248047,0.62500,0.019700,10.45,0.187500,0.589000,0.019400
5,BusyLoop (x86),50,857.8000,0.323242,3.13920,0.068900,50.81,0.271484,3.012500,0.067600
6,BusyLoop (x86),100,1591.6032,0.390625,6.19470,0.117300,100.79,0.332031,5.946500,0.113900
7,SingleStep (AVR),10,270.9000,0.105469,0.06810,0.003383,77.98,0.000000,0.015000,0.000941
8,SingleStep (AVR),50,839.8000,0.294922,0.30620,0.012579,401.78,0.000000,0.072900,0.002039
9,SingleStep (AVR),100,1648.6400,0.158203,0.57840,0.019200,805.39,0.000000,0.138800,0.004000


In [132]:
cpu_mean_baseline = df[df["n"] == 0]["cpu_mean"][0]
df["cpu_overhead_per_session"] = (df["cpu_mean"] - cpu_mean_baseline - df["exec_cpu_mean"]) / df["n"]


oh = df["cpu_overhead_per_session"].values[1:]
print(oh.mean())
print(oh.std())

# profiles = [
#     "SingleStep (x86)",
#     "BusyLoop (x86)",
#     "SingleStep (AVR)",
#     "BusyLoop (AVR)"
# ]

# for profile in profiles:
#     d = df[df["profile"] == profile]
#     mean = ((d.values[:,4] - d.values[:,8] - cpu_mean_baseline) / d.values[:,1]).mean()
#     std = (((d.values[:,5] / d.values[:,1]) ** 2).sum() / (len(d) ** 2))**.5
#     print(profile, mean, std)

0.0036928433333333357
0.0009571840557884103
